## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.7.7"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.7.7

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_SL244FyWARcFdKyTqDizXF2q


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Get the upcoming weekend dates
# Assuming the weekend is Saturday and Sunday
days_ahead = 5 - today.weekday() if today.weekday() <= 4 else 0
saturday = today + timedelta(days=days_ahead)
sunday = saturday + timedelta(days=1)

today, saturday, sunday
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta September 28 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta September 28 2024"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4712253893115949\u0026mkt=en-US\u0026setlang=en-US\u0026w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"\u2666 Atlanta Streets Alive. September 15, 2024 West End to Grant Park FREE. This is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u2014 so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=5012794228343791\u0026mkt=en-US\u0026setlang=en-US\u0026w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Saturday until 09/28/2024. Protect Yo Energy Con 9/22/24 \u2013 11/9/24. ... September 28. Saturday. Sweet Auburn Music Fest 9/28/24 \u2013 9/29/24. Editors\u2019 Pick . October 01.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4819881473017562\u0026mkt=en-US\u0026setlang=en-US\u0026w=jjFkJ7EEAs2P1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food \u0026amp; Wine Festival highlights the best food, wine, beer \u0026amp; cocktails Atlanta has to offer. This unique event brings in 150\u002B chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines \u0026amp; special Tasting Tents.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=4642636758720807\u0026mkt=en-US\u0026setlang=en-US\u0026w=PQg3e5KvVPE_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things To Do This Weekend In London: 28-29 September 2024","url":"https://londonist.com/london/weekend/things-to-do-this-weekend-in-london-28-29-september-2024","displayUrl":"https://londonist.com/.../weekend/things-to-do-this-weekend-in-london-28-29-september-2024","snippet":"All weekend The Classic Car Boot Sale parks up in King\u0026#39;s Cross. SHOREDITCH STREET ART: London Mural Festival is in full swing, with things to see and do all over the city, including at landmark ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=2929911773239\u0026mkt=en-US\u0026setlang=en-US\u0026w=3KmfFj5v5IVHoB4-GHq7gRWSUxzfujXU"},{"name":"Things to Do In and Around New Jersey September 20-22","url":"https://www.njfamily.com/things-to-do-in-and-around-new-jersey-september-20-22/","displayUrl":"https://www.njfamily.com/things-to-do-in-and-around-new-jersey-september-20-22","snippet":"Here are the best things to do this weekend with the family in and around New Jersey, September 20-22, 2024. Search Go. Travel; THINGS TO DO . CALENDAR; FAIRS \u0026amp; FESTIVALS; FAMILY TRAVEL; ... By Jennifer Amato Published: September 19, 2024 September 20, 2024 . BRIDGEWATER COMMONS / Mid-Autumn Lantern Festival.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28\u002B2024\u0026d=1830331120677\u0026mkt=en-US\u0026setlang=en-US\u0026w=VYL0Hh5VDsAeXWmAA-r8La0lBAo32jOr","thumbnailUrl":"https://www.bing.com/th?id=OIF.jQeT3SuiR%2b%2bIlHvRQgdzkg\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta September 29 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta September 29 2024"},"webPages":{"value":[{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4712253893115949\u0026mkt=en-US\u0026setlang=en-US\u0026w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... 29 tickets remaining! Tickets; Sep. 25. 2024. 8:00 PM Wed. 454. Aisle 5 | Capacity: 300. 30307, 1123 Euclid Ave Ne, Atlanta, GA, US","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"September 27th to October 6, 2024 Atlanta Motor Speedway ... September 29, 2024 Lake Sheryl 4540 Lee Rd SW Snellville, GA 30039. Come for soul food, music genres from jazz to reggae, local vendors and more. Save room for the collard green ice cream! Visit the festival website.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=5012794228343791\u0026mkt=en-US\u0026setlang=en-US\u0026w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"Updated September 23, 2024 12:14am EDT Things To Do. FOX 5 Atlanta article. ATLANTA - This is a ... Sept. 28-29; Where: 32 Irwin St. NE, Atlanta; What: A three-day music festival featuring live ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=2173865470069\u0026mkt=en-US\u0026setlang=en-US\u0026w=jxnOnRj04ciRcKNV8UVNHXKEd0PX1nap"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Soak up the last of summer and celebrate the start of fall with the best things to do in Atlanta in September 2024! This month brings comfortable temperatures as summer cools down \u2013 with long, golden days that spill into evening. ... 25\u002B Unique \u0026amp; Fun Date Ideas: Atlanta August 29, 2024; Atlanta Airport: Ultimate Guide for Travelers August 26 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Jazz Festival \u2013 May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15\u002B artists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food \u0026amp; Wine Festival highlights the best food, wine, beer \u0026amp; cocktails Atlanta has to offer. This unique event brings in 150\u002B chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines \u0026amp; special Tasting Tents.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4642636758720807\u0026mkt=en-US\u0026setlang=en-US\u0026w=PQg3e5KvVPE_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. ... 30\u002B Best Cocktail Bars: Atlanta \u0026amp; Beyond September 1, 2024; 25\u002B Unique \u0026amp; Fun Date Ideas: Atlanta August 29, 2024; Atlanta Airport: Ultimate Guide for Travelers August 26, 2024; See Tours Available On Your ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B29\u002B2024\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://deenaqrubetta.pages.dev/wbsyfbe-things-to-do-in-atlanta-september-2024-lvjgugj/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Q1-IywcFZIjlWKI_LcqH_gHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/fun-things-to-do-in-Atlanta-3.jpg","hostPageUrl":"https://deenaqrubetta.pages.dev/wbsyfbe-things-to-do-in-atlanta-september-2024-lvjgugj/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Best-Things-to-Do-in-Atlanta-in-2023.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.M9nQL4kFM4rwNQiRkhAekAHaEo\u0026pid=Api","contentUrl":"https://travellersworldwide.com/wp-content/uploads/2022/05/shutterstock_1039281055.jpg","hostPageUrl":"https://travellersworldwide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3I8aZp7EItFo_du1fS5CVQHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/dc/81/14/dc8114905086f0025ffc20dfd179934b.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/649996158733928577/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IpkJcJSgPJRL3KK_uSZz5gHaLH\u0026pid=Api","contentUrl":"https://priuschat.com/media/best-things-to-do-in-atlanta.20888/full?d=1682071390","hostPageUrl":"https://priuschat.com/media/best-things-to-do-in-atlanta.20888/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8B9LmuRXqgsRTEY6n_RNlAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/06/The-Best-Things-to-Do-in-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-event-calendar/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.WoE56TfoQ63T7_SpwvmGKgHaE8\u0026pid=Api","contentUrl":"https://www.experism.com/explore/wp-content/uploads/2023/02/Atlanta-1550x1034.jpg","hostPageUrl":"https://charlenewdaveen.pages.dev/tthckr-major-events-in-atlanta-2024-nildmjsmr-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.QevNZ-pKmOWUNWz6sfVbwwHaMW\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/e0/9f/6c/e09f6c526c31ff9de373c38930206cd1.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/367184175879819492/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026pid=Api","contentUrl":"https://gosouthatlanta.com/images/atlanta-tree-canopy-social-media.jpg","hostPageUrl":"https://gosouthatlanta.com/things-to-do/events/musicals-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQAAAA\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.pinterest.com/pin/758012181026364916/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://discoveratlanta.com/stories/things-to-do/is-atlanta-a-good-place-to-visit-here-are-the-top-35-reasons/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TAb-J4vHX4UsIX--2psZXwHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero.jpg?is-pending-load=1","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.B9dwp_jKTmtE_Fruodr10QHaE8\u0026pid=Api","contentUrl":"https://www.treksplorer.com/wp-content/uploads/things-to-do-in-atlanta.jpg","hostPageUrl":"https://www.treksplorer.com/things-to-do-in-atlanta-ga-usa/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zco_XrsR7HWXyvmCHgOF7wAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/f_auto,q_auto/v1602606068/newAtlanta.com/Atlanta_OneMusicFest_Crowd/Atlanta_OneMusicFest_Crowd.jpg","hostPageUrl":"https://discoveratlanta.com/things-to-do/nightlife/best-festivals-in-the-atl/"}]}}
```

I called the tool named ConvertUrlToMarkdown with these arguments:
```
{"url":"https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 23, 2024ResetCancelApplyEnd DateSep 25, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 126* Sep 23 2024 7:30 PM Mon B5 City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $60 Avg. price ~ $92 25 tickets remaining! Tickets\r\n* Sep 23 2024 8:00 PM Mon Epik High The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $76 Avg. price ~ $170 Available Tickets: 180 Tickets\r\n* Sep 23 2024 8:00 PM Mon greek Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $32 Avg. price ~ $38 16 tickets remaining! Tickets\r\n* Sep 24 2024 6:00 PM Tue Signs of The Swarm Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $31 Avg. price ~ $38 20 tickets remaining! Tickets\r\n* Sep 24 2024 7:00 PM Tue Kehlani Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $50 Avg. price ~ $110 Available Tickets: 369 Tickets\r\n* Sep 24 2024 7:00 PM Tue Edwin Raphael Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $28 Avg. price ~ $33 20 tickets remaining! Tickets\r\n* Sep 24 2024 7:20 PM Tue Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $14 Avg. price ~ $79 Available Tickets: 1862 Tickets\r\n* Sep 24 2024 7:30 PM Tue Atlanta United II vs. Inter Miami II Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $21 Avg. price ~ $25 6 tickets remaining! Tickets\r\n* Sep 24 2024 7:30 PM Tue Nervosa Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Prices from $73 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Juan Fernando Velasco City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $89 Avg. price ~ $137 50 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Sofi Tukker The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $31 Avg. price ~ $80 Available Tickets: 175 Tickets\r\n* Sep 24 2024 8:00 PM Tue DPR Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $40 Avg. price ~ $67 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Magdalena Bay Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $56 Avg. price ~ $72 17 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Victoria Canal Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $31 Avg. price ~ $36 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Rocco Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $36 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue BEAT - Belew/Vai/Levin/Carey Play 80s King Crimson The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $122 Avg. price ~ $224 3 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $44 Avg. price ~ $78 44 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $34 Avg. price ~ $130 Available Tickets: 118 Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $51 Avg. price ~ $262 Available Tickets: 381 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $9 Avg. price ~ $66 Available Tickets: 2617 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $13 Avg. price ~ $144 Available Tickets: 1222 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $49 Avg. price ~ $89 Available Tickets: 60 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $72 Avg. price ~ $148 40 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed 454 Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $33 Avg. price ~ $38 8 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $41 16 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $83 33 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $94 Avg. price ~ $111 8 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $4 Avg. price ~ $130 Available Tickets: 444 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $65 Avg. price ~ $152 Available Tickets: 389 Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[B5 Upcoming Events: 2 City Winery - Atlanta Sep 23, 2024 7:30 PM](https://atlanta-ga.events/tickets/b5/)[Epik High Upcoming Events: 2 The Tabernacle - GA Sep 23, 2024 8:00 PM](https://atlanta-ga.events/tickets/epik-high/)[greek - Band Upcoming Events: 2 Vinyl - GA Sep 23, 2024 8:00 PM](https://atlanta-ga.events/tickets/greek-band/)[Signs of the Swarm Upcoming Events: 2 Hell at The Masquerade - GA Sep 24, 2024 6:00 PM](https://atlanta-ga.events/tickets/signs-of-the-swarm/)[Kehlani Upcoming Events: 2 Lakewood Amphitheatre Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/kehlani/)[Edwin Raphael Upcoming Events: 2 Altar at Masquerade - GA Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/edwin-raphael/)[New York Mets Upcoming Events: 9 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 18 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Atlanta United II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-united-ii/)[Inter Miami II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/inter-miami-ii/)[Nervosa Upcoming Events: 2 Boggs Social \u0026 Supply Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/nervosa/)[Rocco Upcoming Events: 2 Aisle 5 Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/rocco/)[DPR Live Upcoming Events: 2 Coca-Cola Roxy Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/dpr-live/)[Sofi Tukker Upcoming Events: 2 The Tabernacle - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/sofi-tukker/)[Victoria Canal Upcoming Events: 2 Vinyl - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/victoria-canal/)[Beat - Tribute Show Upcoming Events: 2 The Eastern - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/beat-tribute-show/)[Juan Fernando Velasco Upcoming Events: 2 City Winery - Atlanta Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/juan-fernando-velasco/)[Magdalena Bay Upcoming Events: 2 Variety Playhouse Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/magdalena-bay/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[454 Upcoming Events: 2 Aisle 5 Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/454/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 8 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Starletta Upcoming Events: 2 Vinyl - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/starletta/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named ConvertUrlToMarkdown with these arguments:
```
{"url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in September in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading# Cool Things To Do in September in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished August 23, 2024Last Updated September 11, 2024\r\n\r\nIt\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food.\r\n\r\n## Food Extravaganza\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)\r\nTake a big bite. There is plenty to sample at the Atlanta Food \u0026#038; Wine Festival with more than 200 chefs, sommeliers and mixologists. (Photo by Raftermen Photography)The Atlanta Food \u0026amp; Wine Festival returns to Historic Old Fourth Ward Park for a four-day celebration of incomparable food, wine, beer and craft spirits. More than 200 of the most celebrated chefs, sommeliers, mixologists and industry insiders will offer bites and drinks that showcase the exceptional flavors of the region. The festival runs Sept. 12-15.\r\n\r\n## Burger Bash\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647613/newAtlanta.com/Burger-benefit-resized/Burger-benefit-resized.jpg?_i=AA)\r\nThe team from Fox Bros. BBQ takes to the grill to showcase their burger skills at the Burger Benefit Sept. 22. (Photo courtesy of NFA Burger)Some of the city\u2019s and the nation\u2019s leading chefs including personality Andrew Zimmern; Shai Livi of The Third Space; Billy Kramer of NFA Burger; and Nick Leahy of Vice Kitchen will duke it out for an all-day burger showdown at NFA Burger in Chamblee. The Burger Benefit takes place Sept. 22 and features four burger masters commanding the grill in shifts of 90 minutes. Last year the burger benefit raised more than $53,000 in support of The Giving Kitchen, an Atlanta-based nonprofit for hospitality workers. Tickets start at $30, which includes one burger and a side.\r\n\r\n## Spooky Time\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)\r\nNecrotis is just one of the friendly characters waiting to greet \u2014 and scare \u2014 you at NETHERWORLD. (Photo courtesy of NETHERWORLD)As the haunt season approaches, NETHERWORLD is opening on weekends starting Sept. 20 and 21 and will continue for 34 nights of terror through September, October and November. For its 28th Season of Screams, the attraction is offering two terrifying new haunts, Wake the Dead and Mr. Grendel\u2019s Birthday Party of Horrors. Also returning is Dr. Octane\u2019s Bizarre Beverages featuring colorful, bubbling nonalcoholic drinks served in an optional collectible light-up glass. There\u2019s no shortage of Halloween things to do in Atlanta.\r\n\r\n## Harvest on the Hooch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_790,h_1024,c_scale/v1723648296/newAtlanta.com/Harvest-on-the-hooch/Harvest-on-the-hooch.jpg?_i=AA)\r\nEnjoy getting together with friends at the ninth annual Harvest on the Hooch at Chattahoochee Nature Center. (Photo courtesy of Chattahoochee Nature Center)The Chattahoochee Nature Center is hosting the ninth annual Harvest on the Hooch Sept. 22 to raise awareness for the Nature Center\u2019s Unity Garden, which has supplied more than 50 tons of fresh produce to North Fulton Community Charities food pantry. The event will feature food tastings from local chefs, live bluegrass music, garden games, samples from local craft breweries, face painting, and meet and greets with animals.\r\n\r\n## Hispanic Heritage Month\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647690/newAtlanta.com/hispanic-festival/hispanic-festival.jpg?_i=AA)\r\nEnjoy a cultural experience Sept. 17 for the Fiestas Patrias, celebrating Mexican Independence.Plaza Fiesta on Buford Highway will host a Fiestas Patrias celebration Sept. 17. The Mexican Consulate\u2019s \u201CEl Grito \u2014 Mexican Independence Celebration\u201D has been a local favorite for more than 19 years. Last year more than 60,000 people celebrated. Plan to go and enjoy a cultural experience that embraces heritage, entertainment and community. Celebrate Hispanic Heritage Month around Atlanta.\r\n\r\n## Baldie Con\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)\r\nBaldie Con is an empowerment conference for the bald community.Happening at the JW Marriott Atlanta Buckhead, Baldie Con\u0026nbsp;celebrates, empowers and unites women and children who are bald (Baldies) from alopecia, lupus, cancer or by choice. The three-day conference starts with a meet and greet on Sept. 27. There will be a Baldie Ball with an award ceremony and gala dinner on Saturday night. The weekend wraps up on Sept. 29 with a Sunday Jazz Brunch.\r\n\r\n## It\u2019s so Sweet\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)\r\nAuburn Avenue is where you\u2019ll find the Sweet Auburn Music Fest.The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.\r\n\r\n## Forward Warrior\r\n\r\nThe annual Forward Warrior live art event is happening all day, 9 a.m. until 9 p.m. Saturday, on Wylie Street in Cabbagetown.\r\n\r\n## Celebrate a President\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723649158/newAtlanta.com/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb.jpg?_i=AA)\r\nThe Fox Theatre plans a musical extravaganza to celebrate President Carter\u2019s 100th birthday. (Photo courtesy of Fox Theatre)Jimmy Carter turns 100 on Oct. 1, and the Fox Theatre is celebrating in a big way. Presented by Delta Air Lines, a variety of artists including Chuck Leavell, a Georgia resident and keyboardist for the Rolling Stones, D-Nice, Eric Church, Aren Morris and the Atlanta Symphony Orchestra Chamber Chorus will take the stage Sept. 17 to present a musical extravaganza. The event is a fund-raiser for the Carter Center.\r\n\r\n## Sandy Springs Salutes Creativity\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg?_i=AA)\r\nA couple takes a break from the Sandy Springs Arts Festival to enjoy people watching. (Photo courtesy of Caren West PR)The Sandy Springs Fall Arts Festival returns on Sept. 14-15, featuring more than 150 painters, photographers, sculptors, leather and metal craft persons, glass blowers, jewelers and more. Here are more September (and fall) festivals to visit.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n* Cool Things To Do in October in Atlanta\r\n\r\n## Dragon Con Takes Over\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg?_i=AA)\r\nYou never know who you\u2019ll meet on Peachtree Street during Dragon Con. (Photo courtesy of Dragon Con Photography \u00C2\u00A9 Dragon Con, Inc.)Dragon Con, the worlds largest multimedia, popular culture convention focusing on science fiction and fantasy, gaming, comics, literature, art, music and film in the universe comes to Atlanta and, frankly, takes over the city. Walk the streets and meet thousands of costumed folks participating in the workshops, meet and greets with celebrities, the art show, and comic and pop artist alley. But, the biggest fun comes with the parade down Peachtree Street on Aug. 31. Among the celebrities appearing at Dragon Con, which takes place Aug. 29-Sept. 2, are: Gil Gerard, Erin Grey, Seth Green, Greg Grunberg, Cobie Smulders and Ming-Na Wen.\r\n\r\n## Piedmont Park Celebrates 120 Years\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_1024,c_scale/v1725563202/newAtlanta.com/P4PP-IG-Post/P4PP-IG-Post.png?_i=AA)\r\nParty like it\u2019s 1989 as Piedmont Parks celebrates its 120th birthday and the 35th anniversary of the Piedmont Park Conservatory! The nostalgia-themed party will have food from local restaurants, cocktails, a silent auction and other throwback vibes. All proceeds will support the Piedmont Park Conservatory and its ongoing work to maintain and enhance Atlanta\u2019s most visited public park. Find more information about the event here.\r\n\r\n## College Football is Back\r\n\r\nSeptember, of course, heralds the return of college football season. No matter your alma mater, there are bars all around town, which may show your team\u2019s games, which means you can cheer along with like-minded fans. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Gone Fishin\u2019\r\n\r\nSummer is winding down so it\u2019s the perfect time to drop a line and enjoy a day of fishing. There are several places to fish including Jones Creek Park, Lullwater Park, the Chattahoochee River National Recreation Area, Morgan Falls Overlook Park, Sweetwater Creek State Park, Piedmont Park, Murphey Candler Park and Lake Lanier\r\n\r\n## Try an Omakase Restaurant\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647780/newAtlanta.com/Mujo-Andrew-Thomas-Lee-resized/Mujo-Andrew-Thomas-Lee-resized.jpg?_i=AA)\r\nMuj\u00C5\u008D is one of two Atlanta omakase restaurants to receive a MICHELIN star, but there are several other restaurants that offer this unique culinary experience. (Photo of Muj\u00C5\u008D by Andrew Thomas Lee)Omakase is a Japanese dining experience where guests leave the menu in the hands of the chef and receive a seasonal, creative and unforgettable dinner with a progression of small courses. Atlanta has several omakase restaurants, and two \u2014 Muj\u014D and Hayakawa \u2014 were awarded MICHELIN STARS. But Atlanta has several others that offer amazing experiences as well. Try Omakase Table (West Marietta Street); Nagomiya (West Peachtree Street); Saito (Andrew Young International Boulevard); Taka Sushi (Roswell Road); MF Sushi (North Highland Avenue); Chirori (14th Street); Umi (Peachtree Road); and Brush (Peachtree Road). Brush, in fact, just started a \u201Crestaurant-within-a restaurant\u201D concept by offering a chef-driven sushi omakase-only experience in addition to a regular dining experience.\r\n\r\n## News for Kids\r\n\r\nAtlanta\u2019s must-see attractions are stretching your summer fun \u2013 and dollars \u2013 with a September-only promotion for families. Kids free admission this September is available at most attractions and all offer exciting experiences.\u0026nbsp;Plan your Kids Free September Atlanta discovery outing today. All offers start after Labor Day (Sept. 3-30).\r\n\r\n## Yellow Daisy Festival\r\n\r\nVoted one of the top arts and crafts shows in the nation by Sunshine Artist magazine, the Yellow Daisy Festival at Stone Mountain Park draws more than 400 artists and crafters as well as food vendors, crafter demonstrations, live music and a beer garden. The festival, which takes place Sept. 5-8, is great family fun \u2014 and you can get a head start on holiday shopping.\r\n\r\n## Buckhead Arts Festival\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_819,c_scale/v1723647966/newAtlanta.com/Buckhead-arts-festival/Buckhead-arts-festival.jpg?_i=AA)\r\nThe Buckhead Fine Arts Festival is a great way to sip, snack and shop in the heart of Buckhead. (Photo courtesy of Caren West PR.)Sip, shop and stroll in the heart of Buckhead at the Buckhead Fine Arts Festival Sept. 21-22. Presented by the Atlanta Foundation for Public Spaces, the outdoor event will feature more than 100 painters, photographers, sculptors, metalworkers, glass artists, jewelers and more. Enjoy a variety of food and beverage options from vendors and merchants in Buckhead Village.\r\n\r\n## Inaugural Street Festival\r\n\r\nHosted by Blacks in Technology Foundation, the Atlanta Street Food Festival \u0026amp; Bazaar takes place Sept. 2 at Westside Park, Atlanta\u2019s largest park. The first-time festival is a family-friendly event with music, interactive demos and other entertainment. There\u2019s even a dedicated children\u2019s zone with games, activities and entertainment. Come hungry because it is a gastronomic adventure featuring an array of foods from the city\u2019s finest chefs, food trucks and local eateries. There will be everything from barbecue to gourmet burgers to vegan delicacies.\r\n\r\n## Opera in the Yards\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1723647893/newAtlanta.com/pullman-credit-Smily-Eyes-Media/pullman-credit-Smily-Eyes-Media.jpg?_i=AA)\r\nYes, the Atlanta Opera performs at Pullman Yards. (Photo by Smile Eyes Media)The Atlanta Opera Discoveries series continues the \u201CBoh\u00E8me Project\u201D with two immersive productions at Pullman Yards. Produced by\u0026nbsp;Tomer Zvulun, the project comprises repertory performances of a modern-day\u0026nbsp;\u201CLa boh\u00E8me\u201D \u2014 with the COVID-19 pandemic in place of tuberculosis \u2014 with the Broadway show it inspired,\u0026nbsp;\u201CRent,\u201D which updated Puccini\u2019s story of friendship, passion and art by setting it in the midst of the 1990\u2019s HIV/AIDS crisis. Both performed on the same set, with the action taking place in and around the seated audience, the productions will feature different casts, and musical forces and will alternate nights for most of the run.\r\n\r\n## JapanFest\r\n\r\nCelebrate 50 years of culture, friendship and cultural exchange between Japan and Georgia with JapanFest Sept. 21-22 at Gas South Convention Center in Duluth. Enjoy the mesmerizing beauty of traditional Japanese Rakugo, Kabuki, taiko drumming, martial arts demonstration and workshops in bonsai, calligraphy, ikebana flower design and Kimono fitting. Don\u2019t miss the Japanese Food Court where local restaurants will offer bento boxes, rice bowls, ramen noodles, sushi, yahisoba and shaved ice. In addition there will be three traditional Japanese craftsman giving demonstrations in stone carving, Kumihimo braided cords and tatami weaving.\r\n\r\n## PGA\u2019s TOUR Championship Returns\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1685605842/newAtlanta.com/18_TTC_0921_26792_CCp_cc_20604e9ab6/18_TTC_0921_26792_CCp_cc_20604e9ab6.jpg?_i=AA)\r\nFans of professional golf should head to East Lake Golf Club late August through Sept. 1 for the PGA TOUR Championship.East Lake Golf Club hosts the PGA\u2019s TOUR Championship Aug. 29 through Sept. 1.\r\n\r\nAlways keep an eye on our calendar of events for even more things to do in Atlanta.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[Arts \u0026amp; CultureWe\u2019ll admit it can get hot in Atlanta during July, so here are some cool things to do. Cool Things To Do in July in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-july-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Arts \u0026amp; CultureOctober brings lots of cool things to do in Atlanta, and Halloween is just one. Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

Here are some exciting events and things to do in Atlanta this weekend (September 28-29, 2024):

1. **Sweet Auburn Music Fest**
   - **Date**: September 28-29, 2024
   - **Location**: Auburn Avenue, Atlanta
   - **Details**: Enjoy live music ranging from R&B to gospel, an international food court, national artists, celebrities, and various vendors.
   ![Sweet Auburn Music Fest](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg)

2. **Forward Warrior Live Art Event**
   - **Date**: September 28, 2024
   - **Location**: Wylie Street, Cabbagetown, Atlanta
   - **Details**: Enjoy live art from 9 a.m. to 9 p.m. with artists creating murals and other artworks.
   ![Forward Warrior](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg)

3. **Baldie Con**
   - **Date**: September 27-29, 2024
   - **Location**: JW Marriott Atlanta Buckhead
   - **Details**: This empowerment conference celebrates and unites women and children who are bald due to alopecia, lupus, cancer, or by choice. Events include a Baldie Ball and a Sunday Jazz Brunch.
   ![Baldie Con](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg)

4. **NETHERWORLD Haunted House**
   - **Date**: September 28, 2024 (and ongoing through November)
   - **Location**: Stone Mountain, GA
   - **Details**: Experience two new haunts, "Wake the Dead" and "Mr. Grendel’s Birthday Party of Horrors," as well as Dr. Octane’s Bizarre Beverages.
   ![NETHERWORLD](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg)

5. **Fall Festivals Around Atlanta**
   - **Details**: Various fall festivals are happening around Atlanta, including the Atlanta Streets Alive event and the Sandy Springs Fall Arts Festival.
   ![Fall Festivals](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg)

For more information and a comprehensive list of events, you can visit the [Atlanta Events September 2024](https://atlanta-ga.events/september/) and [Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/) pages.

Enjoy your weekend in Atlanta!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events and things to do in Atlanta this weekend (September 28-29, 2024):

1. **Sweet Auburn Music Fest**
   - **Date**: September 28-29, 2024
   - **Location**: Auburn Avenue, Atlanta
   - **Details**: Enjoy live music ranging from R&B to gospel, an international food court, national artists, celebrities, and various vendors.
   ![Sweet Auburn Music Fest](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg)

2. **Forward Warrior Live Art Event**
   - **Date**: September 28, 2024
   - **Location**: Wylie Street, Cabbagetown, Atlanta
   - **Details**: Enjoy live art from 9 a.m. to 9 p.m. with artists creating murals and other artworks.
   ![Forward Warrior](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg)

3. **Baldie Con**
   - **Date**: September 27-29, 2024
   - **Location**: JW Marriott Atlanta Buckhead
   - **Details**: This empowerment conference celebrates and unites women and children who are bald due to alopecia, lupus, cancer, or by choice. Events include a Baldie Ball and a Sunday Jazz Brunch.
   ![Baldie Con](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg)

4. **NETHERWORLD Haunted House**
   - **Date**: September 28, 2024 (and ongoing through November)
   - **Location**: Stone Mountain, GA
   - **Details**: Experience two new haunts, "Wake the Dead" and "Mr. Grendel’s Birthday Party of Horrors," as well as Dr. Octane’s Bizarre Beverages.
   ![NETHERWORLD](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg)

5. **Fall Festivals Around Atlanta**
   - **Details**: Various fall festivals are happening around Atlanta, including the Atlanta Streets Alive event and the Sandy Springs Fall Arts Festival.
   ![Fall Festivals](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg)

For more information and a comprehensive list of events, you can visit the [Atlanta Events September 2024](https://atlanta-ga.events/september/) and [Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/) pages.

Enjoy your weekend in Atlanta!

In [9]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the dates for the next four weekends
weekends = []
for i in range(1, 5):
    saturday = today + timedelta(days=(5 - today.weekday()) + 7 * (i - 1))
    sunday = saturday + timedelta(days=1)
    weekends.append((saturday.strftime('%Y-%m-%d'), sunday.strftime('%Y-%m-%d')))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 28, 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on September 28, 2024"},"webPages":{"value":[{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B28%2c\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B28%2c\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"More details. \u2013 Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. \u2013 Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B28%2c\u002B2024\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=-nsAOlbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"\u2666 Atlanta Streets Alive. September 15, 2024 West End to Grant Park FREE. This is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours \u2014 so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B28%2c\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"ATLANTA - This is a list of upcoming fall festivals in metro Atlanta and North Georgia: September: North Georgia State Fair. When: ... Sept. 27-28, 2024; ... Taste of Atlanta. When: Oct. 18-20, 2024;","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B28%2c\u002B2024\u0026d=2173865470069\u0026mkt=en-US\u0026setlang=en-US\u0026w=jxnOnRj04ciRcKNV8UVNHXKEd0PX1nap"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on October 5, 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on October 5, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4907344187173452\u0026mkt=en-US\u0026setlang=en-US\u0026w=B_sqEJ0YsMi3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4898908879324774\u0026mkt=en-US\u0026setlang=en-US\u0026w=qXhnhPcHzBqzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=4714289705203876\u0026mkt=en-US\u0026setlang=en-US\u0026w=ZgosuhXzNBJfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 5, 2024 Town Center Park Suwanee FREE admission. This all-day music festival takes place in downtown Suwanee. Admission, parking, and entertainment is free. The BBQ tasting tickets s are $1 each, and each taste requires one to four tickets. You can also buy a full plated barbecue meal. There is also a kids fun area and an artist market.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=5012794228343791\u0026mkt=en-US\u0026setlang=en-US\u0026w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"When: Oct. 21-22, 2024 Where : Candler Park, Atlanta What : A fall festival featuring live music, food trucks, an artist market, a 5K run, and kids\u0026#39; activities.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B5%2c\u002B2024\u0026d=2173865470069\u0026mkt=en-US\u0026setlang=en-US\u0026w=jxnOnRj04ciRcKNV8UVNHXKEd0PX1nap"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on October 12, 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on October 12, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B12%2c\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B12%2c\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B12%2c\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B12%2c\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"When: Oct. 12-15, 2024 Where : Piedmont Park, Atlanta What : One of the largest LGBTQ\u002B pride events in the Southeast, featuring a parade, live performances, food, and family-friendly activities.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B12%2c\u002B2024\u0026d=2173865470069\u0026mkt=en-US\u0026setlang=en-US\u0026w=jxnOnRj04ciRcKNV8UVNHXKEd0PX1nap"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on October 19, 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on October 19, 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 | Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Things to do in Atlanta, GA in October, including upcoming festivals, fall and Halloween events, Pride events, comedy, musicals, plays, races, conventions, neighborhood events, cultural events, and more. ... \u2013 PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 19 tickets remaining! Tickets; Oct. 02. 2024. 8:00 PM Wed. T-Bone Burnett Variety Playhouse | Capacity: 1000. 30307, 1099 Euclid Ave, Atlanta, GA, US","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Upside Down | October 19-20 | Epic Events and Entertainment | Epic Cons","url":"https://www.epiccons.com/events/the-upside-down","displayUrl":"https://www.epiccons.com/events/the-upside-down","snippet":"Experience The Upside Down event on October 19-20, 2024, featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things! Dive into the world of your favorite shows and meet fellow fans! ... The Upside Down (October 19-20) I Was Feeling Festive 2 (December 6-8) 2025 Events. I Was Feeling Epic (April ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BOctober\u002B19%2c\u002B2024\u0026d=4700464207900499\u0026mkt=en-US\u0026setlang=en-US\u0026w=T_o7uCc0DGNZhmTFjyQoH8RhyWcnYm_y","thumbnailUrl":"https://www.bing.com/th?id=OIP.Wm4-6OSBCztRSKoEZnMp7AHaEL\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ConvertUrlToMarkdown with these arguments:
```
{"url":"https://atlanta-ga.events/september/"}
```
and got this result:
```
"Atlanta Events September 2024: Concerts, Shows, Sports...JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)\u00D7Categories[All Events](https://atlanta-ga.events/)[Concerts](https://atlanta-ga.events/concerts/)[Shows](https://atlanta-ga.events/shows/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Sports](https://atlanta-ga.events/sports/)[Family Shows](https://atlanta-ga.events/family-shows/)[Festivals](https://atlanta-ga.events/festivals/)[Opera](https://atlanta-ga.events/opera-tickets/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Concerts\r\n* Shows\r\n* Comedy\r\n* Sports\r\n* More Family Shows Festivals Opera Venues\r\n\r\n* Home\r\n* September\r\n\r\n# Atlanta Events September 2024\r\n\r\nExplore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month. Easily search and select your preferred events in Atlanta with our user-friendly calendar. Secure your tickets with 100% assurance at the lowest prices.\r\n\r\nStart DateSep 23, 2024ResetCancelApplyEnd DateSep 25, 2024CategoryAll EventsConcertsTheaterSportsSearchContentsUpcoming EventsList Of All EventsAtlanta, GA## Upcoming Events \u0026 Tickets\r\n\r\nSelect Your DatesResetCancelApplyUpcoming Events: 126* Sep 23 2024 7:30 PM Mon B5 City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $60 Avg. price ~ $92 25 tickets remaining! Tickets\r\n* Sep 23 2024 8:00 PM Mon Epik High The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $76 Avg. price ~ $170 Available Tickets: 180 Tickets\r\n* Sep 23 2024 8:00 PM Mon greek Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $32 Avg. price ~ $38 16 tickets remaining! Tickets\r\n* Sep 24 2024 6:00 PM Tue Signs of The Swarm Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $31 Avg. price ~ $38 20 tickets remaining! Tickets\r\n* Sep 24 2024 7:00 PM Tue Kehlani Lakewood Amphitheatre | Capacity: 19000 30315, 2002 Lakewood Avenue Southeast, Atlanta, GA, US Prices from $50 Avg. price ~ $110 Available Tickets: 369 Tickets\r\n* Sep 24 2024 7:00 PM Tue Edwin Raphael Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $28 Avg. price ~ $33 20 tickets remaining! Tickets\r\n* Sep 24 2024 7:20 PM Tue Atlanta Braves vs. New York Mets Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $14 Avg. price ~ $79 Available Tickets: 1862 Tickets\r\n* Sep 24 2024 7:30 PM Tue Atlanta United II vs. Inter Miami II Fifth Third Stadium | Capacity: 10200 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Prices from $21 Avg. price ~ $25 6 tickets remaining! Tickets\r\n* Sep 24 2024 7:30 PM Tue Nervosa Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Prices from $73 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Juan Fernando Velasco City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $89 Avg. price ~ $137 50 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Sofi Tukker The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $31 Avg. price ~ $80 Available Tickets: 175 Tickets\r\n* Sep 24 2024 8:00 PM Tue DPR Coca-Cola Roxy | Capacity: 3600 30339, 800 Battery Ave SE, Atlanta, GA, US Prices from $40 Avg. price ~ $67 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Magdalena Bay Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $56 Avg. price ~ $72 17 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Victoria Canal Vinyl - GA 30309, 1374 West Peachtree Street, Atlanta, GA, US Prices from $31 Avg. price ~ $36 16 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue Rocco Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $36 Avg. price ~ $101 15 tickets remaining! Tickets\r\n* Sep 24 2024 8:00 PM Tue BEAT - Belew/Vai/Levin/Carey Play 80s King Crimson The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $122 Avg. price ~ $224 3 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Sevendust The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $44 Avg. price ~ $78 44 tickets remaining! Tickets\r\n* Sep 25 2024 6:30 PM Wed Lorna Shore The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $34 Avg. price ~ $130 Available Tickets: 118 Tickets\r\n* Sep 25 2024 7:00 PM Wed Creed, 3 Doors Down \u0026 Finger Eleven Ameris Bank Amphitheatre | Capacity: 12000 30004, 2200 Encore Parkway, Alpharetta, GA, US Prices from $51 Avg. price ~ $262 Available Tickets: 381 Tickets\r\n* Sep 25 2024 7:20 PM Wed Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $9 Avg. price ~ $66 Available Tickets: 2617 Tickets\r\n* Sep 25 2024 7:30 PM Wed Kings of Leon \u0026 Phantogram State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $13 Avg. price ~ $144 Available Tickets: 1222 Tickets\r\n* Sep 25 2024 8:00 PM Wed Gritz and Jelly Butter City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $74 Avg. price ~ $89 8 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed Lasso - Artist Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $49 Avg. price ~ $89 Available Tickets: 60 Tickets\r\n* Sep 25 2024 8:00 PM Wed Shaboozey Variety Playhouse | Capacity: 1000 30307, 1099 Euclid Ave, Atlanta, GA, US Prices from $72 Avg. price ~ $148 40 tickets remaining! Tickets\r\n* Sep 25 2024 8:00 PM Wed 454 Aisle 5 | Capacity: 300 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Prices from $33 Avg. price ~ $38 8 tickets remaining! Tickets\r\n* Sep 26 2024 6:30 PM Thu Rain City Drive Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $35 Avg. price ~ $41 16 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Mico Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $37 Avg. price ~ $83 33 tickets remaining! Tickets\r\n* Sep 26 2024 7:00 PM Thu Lyfe Jennings City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $94 Avg. price ~ $111 8 tickets remaining! Tickets\r\n* Sep 26 2024 7:20 PM Thu Atlanta Braves vs. New York Mets VIP Packages Truist Park | Capacity: 41149 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Prices from $4 Avg. price ~ $130 Available Tickets: 444 Tickets\r\n* Sep 26 2024 7:30 PM Thu Khruangbin Fabulous Fox Theatre - Atlanta | Capacity: 4678 30305, 660 Peachtree St NE, Atlanta, GA, US Prices from $65 Avg. price ~ $152 Available Tickets: 389 Tickets\r\n\r\nShow More### All Events This Month\r\n\r\n[B5 Upcoming Events: 2 City Winery - Atlanta Sep 23, 2024 7:30 PM](https://atlanta-ga.events/tickets/b5/)[Epik High Upcoming Events: 2 The Tabernacle - GA Sep 23, 2024 8:00 PM](https://atlanta-ga.events/tickets/epik-high/)[greek - Band Upcoming Events: 2 Vinyl - GA Sep 23, 2024 8:00 PM](https://atlanta-ga.events/tickets/greek-band/)[Signs of the Swarm Upcoming Events: 2 Hell at The Masquerade - GA Sep 24, 2024 6:00 PM](https://atlanta-ga.events/tickets/signs-of-the-swarm/)[Kehlani Upcoming Events: 2 Lakewood Amphitheatre Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/kehlani/)[Edwin Raphael Upcoming Events: 2 Altar at Masquerade - GA Sep 24, 2024 7:00 PM](https://atlanta-ga.events/tickets/edwin-raphael/)[New York Mets Upcoming Events: 9 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/new-york-mets/)[Atlanta Braves Upcoming Events: 18 Truist Park Sep 24, 2024 7:20 PM](https://atlanta-ga.events/tickets/atlanta-braves/)[Atlanta United II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/atlanta-united-ii/)[Inter Miami II Upcoming Events: 3 Fifth Third Stadium Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/inter-miami-ii/)[Nervosa Upcoming Events: 2 Boggs Social \u0026 Supply Sep 24, 2024 7:30 PM](https://atlanta-ga.events/tickets/nervosa/)[Rocco Upcoming Events: 2 Aisle 5 Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/rocco/)[DPR Live Upcoming Events: 2 Coca-Cola Roxy Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/dpr-live/)[Sofi Tukker Upcoming Events: 2 The Tabernacle - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/sofi-tukker/)[Victoria Canal Upcoming Events: 2 Vinyl - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/victoria-canal/)[Beat - Tribute Show Upcoming Events: 2 The Eastern - GA Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/beat-tribute-show/)[Juan Fernando Velasco Upcoming Events: 2 City Winery - Atlanta Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/juan-fernando-velasco/)[Magdalena Bay Upcoming Events: 2 Variety Playhouse Sep 24, 2024 8:00 PM](https://atlanta-ga.events/tickets/magdalena-bay/)[Lorna Shore Upcoming Events: 2 The Tabernacle - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/lorna-shore/)[Sevendust Upcoming Events: 2 The Eastern - GA Sep 25, 2024 6:30 PM](https://atlanta-ga.events/tickets/sevendust/)[Creed Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/creed/)[3 Doors Down Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/3-doors-down/)[Finger Eleven Upcoming Events: 2 Ameris Bank Amphitheatre Sep 25, 2024 7:00 PM](https://atlanta-ga.events/tickets/finger-eleven/)[Kings Of Leon Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/kings-of-leon/)[Phantogram Upcoming Events: 2 State Farm Arena - GA Sep 25, 2024 7:30 PM](https://atlanta-ga.events/tickets/phantogram/)[454 Upcoming Events: 2 Aisle 5 Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/454/)[Shaboozey Upcoming Events: 2 Variety Playhouse Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/shaboozey/)[Lasso - Artist Upcoming Events: 2 Buckhead Theatre Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/lasso-artist/)[Gritz and Jelly Butter Upcoming Events: 2 City Winery - Atlanta Sep 25, 2024 8:00 PM](https://atlanta-ga.events/tickets/gritz-and-jelly-butter/)[Rain City Drive Upcoming Events: 2 Hell at The Masquerade - GA Sep 26, 2024 6:30 PM](https://atlanta-ga.events/tickets/rain-city-drive/)[Lyfe Jennings Upcoming Events: 8 City Winery - Atlanta Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/lyfe-jennings/)[Mico Upcoming Events: 2 Purgatory Stage at Masquerade - GA Sep 26, 2024 7:00 PM](https://atlanta-ga.events/tickets/mico/)[Disney On Ice Upcoming Events: 24 Tulsa Expo Square - Pavilion Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/disney-on-ice/)[Khruangbin Upcoming Events: 6 Fabulous Fox Theatre - Atlanta Sep 26, 2024 7:30 PM](https://atlanta-ga.events/tickets/khruangbin/)[The California Honeydrops Upcoming Events: 2 Variety Playhouse Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/the-california-honeydrops/)[Lawrence - Band Upcoming Events: 2 The Tabernacle - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/lawrence-band/)[Nathalie Stutzmann Upcoming Events: 4 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/nathalie-stutzmann/)[Jesus Adrian Romero Upcoming Events: 2 Cobb Energy Performing Arts Centre Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/jesus-adrian-romero/)[Starletta Upcoming Events: 2 Vinyl - GA Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/starletta/)[Atlanta Symphony Orchestra Upcoming Events: 6 Atlanta Symphony Hall Sep 26, 2024 8:00 PM](https://atlanta-ga.events/tickets/atlanta-symphony-orchestra/)Show MoreAtlanta Events Calendar[](https://atlanta-ga.events)[Contact Us](#)Stay up to date! Join our newsletter now to stay up to date on all events in Atlanta plus the occasional special offer and discount code! Epic music events and incredible deals straight to your inbox. Either way, you\u2019ll be the first to know about the best shows in your area.\r\n\r\n[](https://www.facebook.com/)[](https://www.youtube.com/)[](https://www.instagram.com/)[](https://twitter.com/)[](https://www.pinterest.com/)Subscription DealsNameEmail7 \u002B 15 =SubscribeAtlanta-GA.Events|[Sitemap](https://atlanta-ga.events/sitemaps/)|[Privacy Policy](#)|All Rights Reserved 2024"
```

I called the tool named ConvertUrlToMarkdown with these arguments:
```
{"url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in September in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading# Cool Things To Do in September in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished August 23, 2024Last Updated September 11, 2024\r\n\r\nIt\u2019s September. In Atlanta that means it\u2019s time for festivals, costumed aliens walking down Peachtree Street and food \u2014 glorious food.\r\n\r\n## Food Extravaganza\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)\r\nTake a big bite. There is plenty to sample at the Atlanta Food \u0026#038; Wine Festival with more than 200 chefs, sommeliers and mixologists. (Photo by Raftermen Photography)The Atlanta Food \u0026amp; Wine Festival returns to Historic Old Fourth Ward Park for a four-day celebration of incomparable food, wine, beer and craft spirits. More than 200 of the most celebrated chefs, sommeliers, mixologists and industry insiders will offer bites and drinks that showcase the exceptional flavors of the region. The festival runs Sept. 12-15.\r\n\r\n## Burger Bash\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647613/newAtlanta.com/Burger-benefit-resized/Burger-benefit-resized.jpg?_i=AA)\r\nThe team from Fox Bros. BBQ takes to the grill to showcase their burger skills at the Burger Benefit Sept. 22. (Photo courtesy of NFA Burger)Some of the city\u2019s and the nation\u2019s leading chefs including personality Andrew Zimmern; Shai Livi of The Third Space; Billy Kramer of NFA Burger; and Nick Leahy of Vice Kitchen will duke it out for an all-day burger showdown at NFA Burger in Chamblee. The Burger Benefit takes place Sept. 22 and features four burger masters commanding the grill in shifts of 90 minutes. Last year the burger benefit raised more than $53,000 in support of The Giving Kitchen, an Atlanta-based nonprofit for hospitality workers. Tickets start at $30, which includes one burger and a side.\r\n\r\n## Spooky Time\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)\r\nNecrotis is just one of the friendly characters waiting to greet \u2014 and scare \u2014 you at NETHERWORLD. (Photo courtesy of NETHERWORLD)As the haunt season approaches, NETHERWORLD is opening on weekends starting Sept. 20 and 21 and will continue for 34 nights of terror through September, October and November. For its 28th Season of Screams, the attraction is offering two terrifying new haunts, Wake the Dead and Mr. Grendel\u2019s Birthday Party of Horrors. Also returning is Dr. Octane\u2019s Bizarre Beverages featuring colorful, bubbling nonalcoholic drinks served in an optional collectible light-up glass. There\u2019s no shortage of Halloween things to do in Atlanta.\r\n\r\n## Harvest on the Hooch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_790,h_1024,c_scale/v1723648296/newAtlanta.com/Harvest-on-the-hooch/Harvest-on-the-hooch.jpg?_i=AA)\r\nEnjoy getting together with friends at the ninth annual Harvest on the Hooch at Chattahoochee Nature Center. (Photo courtesy of Chattahoochee Nature Center)The Chattahoochee Nature Center is hosting the ninth annual Harvest on the Hooch Sept. 22 to raise awareness for the Nature Center\u2019s Unity Garden, which has supplied more than 50 tons of fresh produce to North Fulton Community Charities food pantry. The event will feature food tastings from local chefs, live bluegrass music, garden games, samples from local craft breweries, face painting, and meet and greets with animals.\r\n\r\n## Hispanic Heritage Month\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647690/newAtlanta.com/hispanic-festival/hispanic-festival.jpg?_i=AA)\r\nEnjoy a cultural experience Sept. 17 for the Fiestas Patrias, celebrating Mexican Independence.Plaza Fiesta on Buford Highway will host a Fiestas Patrias celebration Sept. 17. The Mexican Consulate\u2019s \u201CEl Grito \u2014 Mexican Independence Celebration\u201D has been a local favorite for more than 19 years. Last year more than 60,000 people celebrated. Plan to go and enjoy a cultural experience that embraces heritage, entertainment and community. Celebrate Hispanic Heritage Month around Atlanta.\r\n\r\n## Baldie Con\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_668,h_1024,c_scale/v1724428012/newAtlanta.com/Baldie-Con-flyer/Baldie-Con-flyer.jpeg?_i=AA)\r\nBaldie Con is an empowerment conference for the bald community.Happening at the JW Marriott Atlanta Buckhead, Baldie Con\u0026nbsp;celebrates, empowers and unites women and children who are bald (Baldies) from alopecia, lupus, cancer or by choice. The three-day conference starts with a meet and greet on Sept. 27. There will be a Baldie Ball with an award ceremony and gala dinner on Saturday night. The weekend wraps up on Sept. 29 with a Sunday Jazz Brunch.\r\n\r\n## It\u2019s so Sweet\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_442,c_scale/v1602355972/newAtlanta.com/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled/Zz05MmIyZjQyZWNhOWYxMWVhYjNjMTBhODU2ZjA4Y2UwNw-scaled.jpg?_i=AA)\r\nAuburn Avenue is where you\u2019ll find the Sweet Auburn Music Fest.The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.\r\n\r\n## Forward Warrior\r\n\r\nThe annual Forward Warrior live art event is happening all day, 9 a.m. until 9 p.m. Saturday, on Wylie Street in Cabbagetown.\r\n\r\n## Celebrate a President\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723649158/newAtlanta.com/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb/FoxATL2017_Concert_ALIVECOVERAGE_27323552fb.jpg?_i=AA)\r\nThe Fox Theatre plans a musical extravaganza to celebrate President Carter\u2019s 100th birthday. (Photo courtesy of Fox Theatre)Jimmy Carter turns 100 on Oct. 1, and the Fox Theatre is celebrating in a big way. Presented by Delta Air Lines, a variety of artists including Chuck Leavell, a Georgia resident and keyboardist for the Rolling Stones, D-Nice, Eric Church, Aren Morris and the Atlanta Symphony Orchestra Chamber Chorus will take the stage Sept. 17 to present a musical extravaganza. The event is a fund-raiser for the Carter Center.\r\n\r\n## Sandy Springs Salutes Creativity\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_658,c_scale/v1723648070/newAtlanta.com/sandy-springs-arts-festival/sandy-springs-arts-festival.jpg?_i=AA)\r\nA couple takes a break from the Sandy Springs Arts Festival to enjoy people watching. (Photo courtesy of Caren West PR)The Sandy Springs Fall Arts Festival returns on Sept. 14-15, featuring more than 150 painters, photographers, sculptors, leather and metal craft persons, glass blowers, jewelers and more. Here are more September (and fall) festivals to visit.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n* Cool Things To Do in October in Atlanta\r\n\r\n## Dragon Con Takes Over\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1723647734/newAtlanta.com/Media_Relations23-1024x682-1/Media_Relations23-1024x682-1.jpg?_i=AA)\r\nYou never know who you\u2019ll meet on Peachtree Street during Dragon Con. (Photo courtesy of Dragon Con Photography \u00C2\u00A9 Dragon Con, Inc.)Dragon Con, the worlds largest multimedia, popular culture convention focusing on science fiction and fantasy, gaming, comics, literature, art, music and film in the universe comes to Atlanta and, frankly, takes over the city. Walk the streets and meet thousands of costumed folks participating in the workshops, meet and greets with celebrities, the art show, and comic and pop artist alley. But, the biggest fun comes with the parade down Peachtree Street on Aug. 31. Among the celebrities appearing at Dragon Con, which takes place Aug. 29-Sept. 2, are: Gil Gerard, Erin Grey, Seth Green, Greg Grunberg, Cobie Smulders and Ming-Na Wen.\r\n\r\n## Piedmont Park Celebrates 120 Years\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_1024,c_scale/v1725563202/newAtlanta.com/P4PP-IG-Post/P4PP-IG-Post.png?_i=AA)\r\nParty like it\u2019s 1989 as Piedmont Parks celebrates its 120th birthday and the 35th anniversary of the Piedmont Park Conservatory! The nostalgia-themed party will have food from local restaurants, cocktails, a silent auction and other throwback vibes. All proceeds will support the Piedmont Park Conservatory and its ongoing work to maintain and enhance Atlanta\u2019s most visited public park. Find more information about the event here.\r\n\r\n## College Football is Back\r\n\r\nSeptember, of course, heralds the return of college football season. No matter your alma mater, there are bars all around town, which may show your team\u2019s games, which means you can cheer along with like-minded fans. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Gone Fishin\u2019\r\n\r\nSummer is winding down so it\u2019s the perfect time to drop a line and enjoy a day of fishing. There are several places to fish including Jones Creek Park, Lullwater Park, the Chattahoochee River National Recreation Area, Morgan Falls Overlook Park, Sweetwater Creek State Park, Piedmont Park, Murphey Candler Park and Lake Lanier\r\n\r\n## Try an Omakase Restaurant\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723647780/newAtlanta.com/Mujo-Andrew-Thomas-Lee-resized/Mujo-Andrew-Thomas-Lee-resized.jpg?_i=AA)\r\nMuj\u00C5\u008D is one of two Atlanta omakase restaurants to receive a MICHELIN star, but there are several other restaurants that offer this unique culinary experience. (Photo of Muj\u00C5\u008D by Andrew Thomas Lee)Omakase is a Japanese dining experience where guests leave the menu in the hands of the chef and receive a seasonal, creative and unforgettable dinner with a progression of small courses. Atlanta has several omakase restaurants, and two \u2014 Muj\u014D and Hayakawa \u2014 were awarded MICHELIN STARS. But Atlanta has several others that offer amazing experiences as well. Try Omakase Table (West Marietta Street); Nagomiya (West Peachtree Street); Saito (Andrew Young International Boulevard); Taka Sushi (Roswell Road); MF Sushi (North Highland Avenue); Chirori (14th Street); Umi (Peachtree Road); and Brush (Peachtree Road). Brush, in fact, just started a \u201Crestaurant-within-a restaurant\u201D concept by offering a chef-driven sushi omakase-only experience in addition to a regular dining experience.\r\n\r\n## News for Kids\r\n\r\nAtlanta\u2019s must-see attractions are stretching your summer fun \u2013 and dollars \u2013 with a September-only promotion for families. Kids free admission this September is available at most attractions and all offer exciting experiences.\u0026nbsp;Plan your Kids Free September Atlanta discovery outing today. All offers start after Labor Day (Sept. 3-30).\r\n\r\n## Yellow Daisy Festival\r\n\r\nVoted one of the top arts and crafts shows in the nation by Sunshine Artist magazine, the Yellow Daisy Festival at Stone Mountain Park draws more than 400 artists and crafters as well as food vendors, crafter demonstrations, live music and a beer garden. The festival, which takes place Sept. 5-8, is great family fun \u2014 and you can get a head start on holiday shopping.\r\n\r\n## Buckhead Arts Festival\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_819,c_scale/v1723647966/newAtlanta.com/Buckhead-arts-festival/Buckhead-arts-festival.jpg?_i=AA)\r\nThe Buckhead Fine Arts Festival is a great way to sip, snack and shop in the heart of Buckhead. (Photo courtesy of Caren West PR.)Sip, shop and stroll in the heart of Buckhead at the Buckhead Fine Arts Festival Sept. 21-22. Presented by the Atlanta Foundation for Public Spaces, the outdoor event will feature more than 100 painters, photographers, sculptors, metalworkers, glass artists, jewelers and more. Enjoy a variety of food and beverage options from vendors and merchants in Buckhead Village.\r\n\r\n## Inaugural Street Festival\r\n\r\nHosted by Blacks in Technology Foundation, the Atlanta Street Food Festival \u0026amp; Bazaar takes place Sept. 2 at Westside Park, Atlanta\u2019s largest park. The first-time festival is a family-friendly event with music, interactive demos and other entertainment. There\u2019s even a dedicated children\u2019s zone with games, activities and entertainment. Come hungry because it is a gastronomic adventure featuring an array of foods from the city\u2019s finest chefs, food trucks and local eateries. There will be everything from barbecue to gourmet burgers to vegan delicacies.\r\n\r\n## Opera in the Yards\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1723647893/newAtlanta.com/pullman-credit-Smily-Eyes-Media/pullman-credit-Smily-Eyes-Media.jpg?_i=AA)\r\nYes, the Atlanta Opera performs at Pullman Yards. (Photo by Smile Eyes Media)The Atlanta Opera Discoveries series continues the \u201CBoh\u00E8me Project\u201D with two immersive productions at Pullman Yards. Produced by\u0026nbsp;Tomer Zvulun, the project comprises repertory performances of a modern-day\u0026nbsp;\u201CLa boh\u00E8me\u201D \u2014 with the COVID-19 pandemic in place of tuberculosis \u2014 with the Broadway show it inspired,\u0026nbsp;\u201CRent,\u201D which updated Puccini\u2019s story of friendship, passion and art by setting it in the midst of the 1990\u2019s HIV/AIDS crisis. Both performed on the same set, with the action taking place in and around the seated audience, the productions will feature different casts, and musical forces and will alternate nights for most of the run.\r\n\r\n## JapanFest\r\n\r\nCelebrate 50 years of culture, friendship and cultural exchange between Japan and Georgia with JapanFest Sept. 21-22 at Gas South Convention Center in Duluth. Enjoy the mesmerizing beauty of traditional Japanese Rakugo, Kabuki, taiko drumming, martial arts demonstration and workshops in bonsai, calligraphy, ikebana flower design and Kimono fitting. Don\u2019t miss the Japanese Food Court where local restaurants will offer bento boxes, rice bowls, ramen noodles, sushi, yahisoba and shaved ice. In addition there will be three traditional Japanese craftsman giving demonstrations in stone carving, Kumihimo braided cords and tatami weaving.\r\n\r\n## PGA\u2019s TOUR Championship Returns\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1024,h_682,c_scale/v1685605842/newAtlanta.com/18_TTC_0921_26792_CCp_cc_20604e9ab6/18_TTC_0921_26792_CCp_cc_20604e9ab6.jpg?_i=AA)\r\nFans of professional golf should head to East Lake Golf Club late August through Sept. 1 for the PGA TOUR Championship.East Lake Golf Club hosts the PGA\u2019s TOUR Championship Aug. 29 through Sept. 1.\r\n\r\nAlways keep an eye on our calendar of events for even more things to do in Atlanta.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[Arts \u0026amp; CultureWe\u2019ll admit it can get hot in Atlanta during July, so here are some cool things to do. Cool Things To Do in July in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-july-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Arts \u0026amp; CultureOctober brings lots of cool things to do in Atlanta, and Halloween is just one. Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

I called the tool named ConvertUrlToMarkdown with these arguments:
```
{"url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/"}
```
and got this result:
```
"Cool Things To Do in October in Atlanta - Discover Atlanta[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\nMenu* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\nCheck out these storiesLoading\u0022The Lion King\u0022 (Photo by Joan Marcus)\r\n\r\n# Cool Things To Do in October in Atlanta\r\n\r\nBy Mary Welch\r\n\r\nPublished September 13, 2024Last Updated September 19, 2024\r\n\r\nThe weather has cooled off and Atlanta is ready to embrace the fall with new exhibits, fashion shows and festivals \u2014 lots and lots of festivals. October always brings Atlanta Pride. Prepare with our Pride Survival Guide. Hispanic Heritage Month continues through Oct. 15.\r\n\r\n## The Lion King\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1706,c_scale/v1726166160/newAtlanta.com/S9-GeraldRamseyasMufasa.TheLionKingNorthAmericanTourC2A9Disney.PhotobyMatthewMurphy28229/S9-GeraldRamseyasMufasa.TheLionKingNorthAmericanTourC2A9Disney.PhotobyMatthewMurphy28229.jpg?_i=AA)\r\nGerald Ramsey portrays Mufasa in \u201CThe Lion King\u201D National Tour. (Photo by Matthew Murphy)The Regions Bank Broadway in Atlanta series starts up again and leads with the very popular \u201CThe Lion King\u201D Oct. 2 through Oct. 20 at the Fox Theatre. \u201CThe Lion King\u201D toured North America for more than 22 years, and during that time welcomed more than 23 million theater goers, making it North America\u2019s longest-running and most-attended Broadway tour. This lavish production features a score by Elton John, Tim Rice and additional songs by Lego M, Mark Mancina, Jay Rifkin, Julie Taylor and Hans Zimmer. \u201CThe Lion King\u201D is sure to dazzle theater lovers of all ages.\r\n\r\n## Jazz it Up\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166194/newAtlanta.com/Hammonds-House-Museum-photo-credit-Karen-Hatchett-copy/Hammonds-House-Museum-photo-credit-Karen-Hatchett-copy.jpg?_i=AA)\r\nGet jazzed at Hammonds House Museum. (Photo by Karen Hatchett)An exciting weekend of jazz will take place at the Hammonds House Museum as trombonist Wycliffe Garden and pianist Eric Scott Reed perform Oct. 5 and present a workshop for musicians and jazz lovers on Oct. 6. A veteran member of the Wynton Marsalis Septet and an original member of Lincoln Center Jazz Orchestra, Gordon has had an impressive career touring the world performing hard-swinging, straight-ahead jazz\u0026nbsp;while receiving great acclaim from audiences and critics alike. Eric Scott Reed has been a vital force in jazz for more than 25 years, excelling as a leader, composer, producer, educator, sideman and member of\u0026nbsp;the Marsalis\u2019 Septet and Lincoln Center Jazz Orchestra.\r\n\r\n## Get a Room\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166250/newAtlanta.com/Hotel-Granada_27893b8e55/Hotel-Granada_27893b8e55.jpg?_i=AA)\r\nThe reimagined Granada Hotel sits just across from the Woodruff Arts Center. (Photo by Grady McGill)Atlanta has amazing hotel rooms \u2014 something for everyone \u2014 and a few more hotels just opened. Hotel Granada, across from the Arts Center MARTA station and the Woodruff Arts Center, was built in 1923 and fell on some hard times. But it has been lovingly reimaged with contemporary Southern flair. At the heart of the 120-room hotel is Pom Court, an open-air oasis that is perfect for hotel guests, locals and museum goers (the High Museum of Art is nearby) to sit on the patio and enjoy a cocktail or quick bite.\r\n\r\nThe Forth Hotel just opened by the Beltline in the Historic Fourth Ward. The hotel features 157 guest rooms and suites as well as 39 residential suites, elevated restaurants and bars. Guests will enjoy an outdoor pool and state-of-the-art wellness amenities, including a 20,000-square-foot fitness center, sauna, steam room, leg compression sleeves, massage guns and vibrating massage balls, and foam rollers.\r\n\r\nIn November, the Origin Hotel Atlanta will welcome guests to its \u201950s-era office building-turned-boutique hotel. The hotel features 122 guest rooms and suites plus Butter \u002B Scotch restaurant and bar, a local retail store front and event space. The Wyndham Hotel is located on Pryor Street, just steps away from the Georgia World Congress Center, Mercedes-Benz Stadium and Centennial Olympic Park.\r\n\r\n## There\u2019s Renoir, Degas, Monet \u2014 and More\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1440,c_scale/v1726166729/newAtlanta.com/Interior-first-exhibition-\u00C2\u00A9-Excurio-GEDEON-Experiences-Musee-dOrsay-copy/Interior-first-exhibition-\u00C2\u00A9-Excurio-GEDEON-Experiences-Musee-dOrsay-copy.jpg?_i=AA)\r\nTravel back to the 19th century at the Eclipse Centre. (Photo courtesy of Excurio-GEDEON Experiences Mus\u00C3\u00A9e)\u201CTonight with the Impressionists\u201D is a virtual reality exhibit coming to the Eclipse Centre that will take you back to 1874 to meet the famous painters of that period. Created in collaboration with the renowned Mus\u00E9e d\u2019Orsay in Paris and in honor of the 150th anniversary of the first-ever Impressionist exhibition, the 45-minute journey transports you back to 1874 when a group of artists broke away from traditional academic painting, focusing instead on capturing light, color and atmosphere in new ways. Known as the Impressionists\u2014Monet, Renoir, Morisot, Degas and others\u2014they depicted everyday life and outdoor scenes with spontaneous brushstrokes and vibrant colors. The exhibition runs through December.\r\n\r\n## Little Five Points Parade\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)\r\nYou might have way too much fun at the Little Five Points Halloween Festival and Parade. (Photo by Reid Koski)Yes, we\u2019ve all seen Halloween parades, but you really haven\u2019t seen a Halloween parade until you venture down to Little Five Points. The bohemian neighborhood sponsors a Halloween Festival on Oct. 19 and 20 that features live music from the city\u2019s best underground acts, an art and vintage market, ghost tours, a skate park, and food and drink vendors. But it\u2019s the parade \u2014 the biggest and spookiest in the Southeast \u2014 that makes it a signature fall event. Don\u2019t believe us \u2014 a number of national publications such as Country Living and Fodors \u2014 have named it among the best Halloween parades in the country. For more Halloween happenings in Atlanta check our Spooktacular Guide to Halloween in Atlanta and our Grown-Up Guide to Halloween in Atlanta.\r\n\r\n## Dragons and the Atlanta Symphony Orchestra\r\n\r\nDreamWorks\u2019 \u201CHow to Train a Dragon\u201D comes to the Woodruff Arts Center Oct. 4 and 5, with a captivating and original story that combines humor, fire-breathing action and epic adventure.\u0026nbsp; Hiccup is a young Viking who defies tradition when he befriends one of his deadliest foes \u2013a ferocious dragon he calls Toothless. Together, these unlikely heroes must fight against all odds to save both their worlds.\r\n\r\n\u201CHow To Train Your Dragon\u201D\u2013In Concert features this acclaimed film presented in HD, with composer John Powell\u2019s Academy Award-nominated score performed live to picture by a full symphony orchestra.\r\n\r\nAlso, don\u2019t miss the Atlanta Symphony Orchestra\u2019s performances of Haydn and Mendelssohn Symphonies Oct. 17-19 and the return Georgia native Roderick Cox who will dig into the rich sounds of the ASO string section for Barber\u2019s soul-stirring \u201CAdagio.\u201D He follows with music from John Adams\u2019 white-knuckle opera\u0026nbsp;\u201CDoctor Atomic\u201D and concludes with the Atlanta Symphony Orchestra going full throttle in Rachmaninoff\u2019s seductive orchestral romp\u0026nbsp;\u201CSymphonic Dances.\u201D\r\n\r\n## Let\u2019s Brunch a Bunch\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166331/newAtlanta.com/Ela-Brunch/Ela-Brunch.jpg?_i=AA)\r\nPut a Mediterranean twist on your brunch at Ela. (Photo by Grady McGill)The weather is cooling off so it\u2019s a perfect time for brunching with friends. Several restaurants are offering brunch options. Ela offers a Mediterranean twist with Turkish hash or souvlaki and eggs. Snooze offers a fall menu that includes pumpkin pecan pie pancakes, pumpkin martinis and a jalape\u00F1o and cheddar omelet.\r\n\r\nIndaco started a weekend brunch with dishes such as breakfast pizza, ricotta toast and eggs in purgatory. The Flying Biscuit always offers a great bunch menu; fall items include cinnamon roll pancakes and a bacon jam burger bowl.\r\n\r\n## Stylin\u2019\r\n\r\nAtlanta Fashion Week (ATLFW) makes a highly anticipated return Oct. 2-7, bringing together a global array of designers and brands to celebrate fashion, art, culture and creativity. This year\u2019s ATLFW promises to be a truly international affair, showcasing renowned talents from Nigeria, the United Kingdom, Senegal and New York, to name a few. Throughout the week, fashion will take center stage for immersive experiences, complete with pop-up showroom displays, brand activations and runway shows.\r\n\r\n## Corn Dogs, Rides and Whack a Mole\r\n\r\nThe Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3.\r\n\r\nRead More* Cool Things to Do in November in Atlanta\r\n* Cool Things To Do in September in Atlanta\r\n* Cool Things To Do in August in Atlanta\r\n\r\n## Pickleball Tournament\r\n\r\nIt\u2019s Pickle Palooza. The Collective Learning Academy of Atlanta is sponsoring a fund-raiser that promises an evening of pickleball action for players of all levels. Taking place Oct. 1 at the Atlanta Pickleball Center, the round robin matches will begin at 6 and go until the knockout round around 9 p.m. There will be light appetizers and beverages. Want to get tournament ready? Head over to Painted Pickle for a few practice rounds.\r\n\r\n## Nando\u2019s African Diaspora Art Comes to Atlanta\r\n\r\n![](https://res.cloudinary.com/atlanta/images/w_1815,h_2560,c_scale/v1726166411/newAtlanta.com/Fisani-Parktown-by-Zenele-Muholi-copy/Fisani-Parktown-by-Zenele-Muholi-copy.jpg?_i=AA)\r\nFisani, Parktown by Zenele MuholiA world-class exhibition curated from\u0026nbsp;Nando\u2019s art collection, \u201CIf You Look Hard Enough, You Can See Our Future,\u201D is making a stop in Atlanta. The world-class exhibition will feature nearly 30 pieces at the African Diaspora Art Museum of Atlanta (ADAMA).\r\n\r\nOpen Sept. 20 through Nov. 2, the exhibition will feature the works of African artists including Zanele Muholi,\u0026nbsp;Portia Zvahera\u0026nbsp;and\u0026nbsp;Mbongeni Buthelizi.\u0026nbsp;\r\n\r\nNando\u2019s, whose restaurants serve South-African flamed-grilled Peri-Peri chicken and Peri-Peri sauce,\u0026nbsp;owns one of the world\u2019s largest collections of Southern African contemporary art and design. The work is displayed in its restaurants globally, including in Atlanta.\r\n\r\n## Holographic Theater Comes to Ponce City Market\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166509/newAtlanta.com/Verse-Immersive-Headset_Photo-Courtesy-of-Verse-Immersive-resized/Verse-Immersive-Headset_Photo-Courtesy-of-Verse-Immersive-resized.png?_i=AA)\r\nTry an augmented reality experience at Verse Immersive. (Photo courtesy of Verse Immersive)A new augmented reality experience, Verse Immersive, is open at The Roof at Ponce City Market. The only holographic theater of its kind, Verse Immersive uses AR visors to bring 3D characters and stories to life. Verse Immersive offers guests an immersive experience of interactive tales. Guests can expect a 30-, 60-, or 90-minute immersive journey during which AR visors will put them at the center of an interactive and engaging story. AR, unlike virtual reality, allows guests to roam freely throughout the room while still seeing and interacting with their surroundings. Five experiences are offered at Verse Immersive and all are appropriate for ages 7 and older.\r\n\r\n## Festivals, Festivals and More Festivals\r\n\r\nFall is the perfect time to explore neighborhoods, hear local musicians, purchase the works of artists and have fun. Yes, it\u2019s festival time and no matter which direction you go, there\u2019s a festival. Among our favorites are the Fall Festival on Ponce, Candler Park Fall Festival, Old Fourth Ward Arts Fest and the Scottish Festival and Highland Games at Stone Mountain Park. ONE MusicFest turns the sound up in Piedmont Park Oct. 26 and 27 with its brand of urban progressive music. Among the performers will be Cardi B, Earth, Wind and Fire, T.I. and Keyshia Cole. OYE Festival on Oct. 6 provides an opportunity to continue celebrating Hispanic Heritage Month. It is a day filled with music, art installations, food and interactive experiences in the heart of Atlanta in Old Fourth Ward. Oct. 13 brings Latinx Art Exhibit to the Decatur Arts Alliance Gallery.\r\n\r\n## A Gamer\u2019s Dream\r\n\r\nDreamHack comes to Atlanta Oct. 4-6 with a weekend full of esports, cosplay, creator-led activities, table top, special guests, panels, workshops, IRL loot stores and, oh yes, gaming. Among the fun will be Brennan Lee Mulligan\u2019s D\u0026amp;D One Shot, the Call of Duty: Mobile World Championship and a cosplay contest.\r\n\r\n## Happy Anniversary\r\n\r\n![](https://res.cloudinary.com/atlanta/images/v1726166637/newAtlanta.com/IMG_0701/IMG_0701.jpg?_i=AA)\r\nMmmm. Munch a grilled skewer of tasty bites at Yakitori Kona. (Photo by Grady McGill)Let\u2019s blow out the candles as several of our favorite hotels and restaurants celebrate another year of pleasing visitors and locals with great food and hospitality. Celebrating first years are Yakitori Kona, an intimate izakaya cocktail bar offering grilled skewers in Virginia Highland, and Verdure, a Midtown restaurant that fuses the flavors and diverse cooking techniques of western and African countries.\r\n\r\nKrog Street Market, the acclaimed food court, celebrates its 10th anniversary while the Hotel Clermont takes the cake with its 100th anniversary. Here\u2019s to many more.\r\n\r\n## Light up Cancer\r\n\r\nThe Leukemia \u0026amp; Lymphoma Society\u2019s Light The Night brings light to the darkness of cancer by supporting lifesaving research, advocacy and services for people battling blood cancer. Join family and friends on Oct. 5 at Piedmont Park to celebrate, honor or remember those affected by blood cancers. Registration is free, but participants are encouraged to raise funds to support the mission. All registered participants will receive an illuminated lantern. In addition to the walk, the evening will be filled with music, fireworks, a kid zone and empowering ceremonies honoring survivors and remembering those lost.\r\n\r\n## Take a Stroll\r\n\r\nAtlanta Streets Alive welcomes walkers, skaters, bicycle riders and cartwheel turners to Ralph David Abernathy Boulevard and Georgia Avenue from 1 until 5 p.m. Oct. 20 when the streets will be closed to vehicular traffic and open to pedestrians. Walk on, Atlanta.\r\n\r\n## It\u2019s Fall: Time for College Football\r\n\r\nFall sports mean one thing in particular in the South: college football. Whatever your alma mater, chances are there\u2019s a bar in Atlanta where you can join like-minded fans to cheer on your team. Find out Where to Watch College Football in Atlanta.\r\n\r\n## Fashion Forward\r\n\r\nFashionFest, which takes place Oct. 11-13, is an immersive ever-evolving platform giving attendees an insider\u2019s look into the world of urban and pop culture by combining elements of a festival and an exhibition and bringing together diverse aspects of the business fashion world.\r\n\r\nFashionFest offers attendees the opportunity to engage with and learn from some of the most fascinating brands and business minds in urban and pop fashion culture, while enjoying live musical performances, exploring the curated marketplace to shop limited edition goods and experiencing cutting-edge technology and sourcing while learning business principles to grow and expand your fashion brands.\r\n\r\nTo find things to do in Atlanta any time, visit our calendar of events.\r\n\r\n![](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/pixel.png)\r\nJournalist Mary Welch writes business and lifestyle stories for local and national publications.\r\n\r\n## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[Arts \u0026amp; CultureSeptember is a great time to be in Atlanta. There are amazing festivals \u2014 food and music \u2014 Black Pride events and football. Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)[AttractionsNovember in Atlanta brings popular festivals, a new Cirque du Soleil show and the beginning of holiday things to do. Cool Things to Do in November in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-november-in-atlanta/)[Arts \u0026amp; CultureIn May, Atlantans head outdoors to enjoy the sun, flowers and the diversity of things to do. Cool Things To Do in May in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-may-in-atlanta/)[Arts \u0026amp; CultureEnjoy all that Atlanta has to offer during the awesome month of April. Cool Things To Do in April in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-april-in-atlanta/)[Arts \u0026amp; CultureFrom outdoor activities to holiday shows, there\u2019s something cool for everyone during December in Atlanta. Cool Things To Do in December in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta/)[Things to doFrom end of summer festivals to new exhibits at the Atlanta History Center, discover cool things to do in August in Atlanta. Cool Things To Do in August in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-august-in-atlanta/)## Sign up for email updates\r\n\r\nEmail AddressZip Code* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\nSelect site languageSelect languageEnglishEspa\u00F1olFran\u00E7aisDeutschPortugu\u00EAs\u4E2D\u6587Submit[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\nAtlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

Here are some exciting activities and events happening in Atlanta for each of the next four weekends:

### September 28-29, 2024
1. **Sweet Auburn Music Fest**: Held on Auburn Avenue, this festival offers live music ranging from R&B to gospel and alternative favorites, international food court, national artists, celebrities, and various vendors.
   - ![Sweet Auburn Music Fest](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

2. **Atlanta Food & Wine Festival**: A four-day celebration featuring over 200 chefs, sommeliers, and mixologists offering food and drinks that showcase regional flavors.
   - ![Atlanta Food & Wine Festival](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

3. **NETHERWORLD Haunted House**: Experience two new terrifying haunts, Wake the Dead and Mr. Grendel’s Birthday Party of Horrors, starting from September 20.
   - ![NETHERWORLD](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

### October 5-6, 2024
1. **Festival on Ponce**: Arts festival in Olmsted Linear Park with over 125 vendors of fine art, folk art, food and drink, and children's activities.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

2. **Candler Park Fall Fest**: A fall festival featuring live music, food trucks, an artist market, and kids' activities.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

3. **Suwanee Fest**: An all-day music festival in downtown Suwanee with free admission, BBQ tasting tickets, a kids fun area, and an artist market.
   - [Read more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

### October 12-13, 2024
1. **Atlanta Pride Festival**: One of the largest LGBTQ+ pride events in the Southeast, featuring a parade, live performances, food, and family-friendly activities in Piedmont Park.
   - ![Atlanta Pride Festival](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

2. **Georgia Apple Festival**: Held in Ellijay, GA, this festival features arts, crafts, apple-based food items, and more. It spans two weekends in October.
   - [Read more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

3. **Atlanta Fair**: Traditional fair with rides, games, and food, located across from the Georgia State University Stadium.
   - ![Atlanta Fair](https://res.cloudinary.com/atlanta/images/v1726166331/newAtlanta.com/Ela-Brunch/Ela-Brunch.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

### October 19-20, 2024
1. **Little Five Points Halloween Festival and Parade**: Live music, art and vintage market, ghost tours, a skate park, and the biggest Halloween parade in the Southeast.
   - ![Little Five Points Halloween Festival](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

2. **PNC Atlanta 10 Miler & 5K**: Annual run events through Buckhead, starting from Atlantic Station.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

3. **The Upside Down Event**: Featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities of "Stranger Things."
   - [Read more](https://www.epiccons.com/events/the-upside-down)

These events provide a variety of activities from music festivals, food fairs, cultural celebrations, to Halloween festivities, ensuring there is something for everyone to enjoy in Atlanta over the next four weekends.


In [10]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here are some exciting activities and events happening in Atlanta for each of the next four weekends:

### September 28-29, 2024
1. **Sweet Auburn Music Fest**: Held on Auburn Avenue, this festival offers live music ranging from R&B to gospel and alternative favorites, international food court, national artists, celebrities, and various vendors.
   - ![Sweet Auburn Music Fest](https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

2. **Atlanta Food & Wine Festival**: A four-day celebration featuring over 200 chefs, sommeliers, and mixologists offering food and drinks that showcase regional flavors.
   - ![Atlanta Food & Wine Festival](https://res.cloudinary.com/atlanta/images/w_1024,h_683,c_scale/v1723648163/newAtlanta.com/ATL-food-Credit-raftermen-photography-copy/ATL-food-Credit-raftermen-photography-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

3. **NETHERWORLD Haunted House**: Experience two new terrifying haunts, Wake the Dead and Mr. Grendel’s Birthday Party of Horrors, starting from September 20.
   - ![NETHERWORLD](https://res.cloudinary.com/atlanta/images/w_684,h_1024,c_scale/v1723647650/newAtlanta.com/NeteherworldMortis-copy/NeteherworldMortis-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

### October 5-6, 2024
1. **Festival on Ponce**: Arts festival in Olmsted Linear Park with over 125 vendors of fine art, folk art, food and drink, and children's activities.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

2. **Candler Park Fall Fest**: A fall festival featuring live music, food trucks, an artist market, and kids' activities.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

3. **Suwanee Fest**: An all-day music festival in downtown Suwanee with free admission, BBQ tasting tickets, a kids fun area, and an artist market.
   - [Read more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

### October 12-13, 2024
1. **Atlanta Pride Festival**: One of the largest LGBTQ+ pride events in the Southeast, featuring a parade, live performances, food, and family-friendly activities in Piedmont Park.
   - ![Atlanta Pride Festival](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

2. **Georgia Apple Festival**: Held in Ellijay, GA, this festival features arts, crafts, apple-based food items, and more. It spans two weekends in October.
   - [Read more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

3. **Atlanta Fair**: Traditional fair with rides, games, and food, located across from the Georgia State University Stadium.
   - ![Atlanta Fair](https://res.cloudinary.com/atlanta/images/v1726166331/newAtlanta.com/Ela-Brunch/Ela-Brunch.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

### October 19-20, 2024
1. **Little Five Points Halloween Festival and Parade**: Live music, art and vintage market, ghost tours, a skate park, and the biggest Halloween parade in the Southeast.
   - ![Little Five Points Halloween Festival](https://res.cloudinary.com/atlanta/images/w_2560,h_1707,c_scale/v1726166287/newAtlanta.com/23-Halloween-Reid-Koski_4-copy/23-Halloween-Reid-Koski_4-copy.jpg?_i=AA)
   - [Read more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

2. **PNC Atlanta 10 Miler & 5K**: Annual run events through Buckhead, starting from Atlantic Station.
   - [Read more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)

3. **The Upside Down Event**: Featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities of "Stranger Things."
   - [Read more](https://www.epiccons.com/events/the-upside-down)

These events provide a variety of activities from music festivals, food fairs, cultural celebrations, to Halloween festivities, ensuring there is something for everyone to enjoy in Atlanta over the next four weekends.

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
